In [1]:
import pandas  as pd
import json
from nltk import ngrams
#from  datasketch.experimental.aio.lsh import AsyncMinHashLSH
#from  datasketch import MinHash, MinHashLSH

In [2]:
articles = pd.read_json("data/assignment3_aricles.json", orient='records', encoding="utf-8")

In [3]:
import nltk
from nltk.util import ngrams
articles.head()

,Content,Title,article_id
0,Are you ready to go back to the Moon? NASA to...,LAMP Educational Site,0
1,WASHINGTON - US Secretary of State Hillary Cli...,Clinton urges Egypt Israel to talk on Sinai,1
2,"The ""hurricane ride-out team"" at NASA's Kenned...",SPACE.com -- Bracing for Impact,2
3,Search the history of over 284 billion web pag...,Internet Archive Wayback Machine,3
4,The Islamic State (IS) Sunni radical group ove...,IS overruns parts of Unesco-listed Syrian city,4


In [4]:
#You can use n-gram at word level for this task
#try with different n-gram values 
# You can use ngrams from nltk for this
def getNgrams(articles):
    n_grams = []
    #return articles with a field ngrams
    #articles['Content'] = articles['Content'].map(lambda x: x.lstrip("')(-?:/.,").rstrip('aAbBcC'))
    for content in articles['Content']:
        n = 2
        content = content.split(' ')
        content = [x.strip().replace('-', '') for x in content]
        content = [x.strip().replace('?', '') for x in content]
        content = [x.strip().replace(':', '') for x in content]
        content = [x.strip().replace('/', '') for x in content]
        content = [x.strip().replace('.', '') for x in content]
        content = [x.strip().replace(',', '') for x in content]
        content = [x.strip().replace('\\', '') for x in content]
        content = [x.strip().replace('(', '') for x in content]
        content = [x.strip().replace(')', '') for x in content]
        content = [x.strip().replace("'", '') for x in content]
        content = [x.strip().replace('"', '') for x in content]
        word_grams = []
        ngram = ngrams(content,n)
        word_grams.append(' '.join(str(i) for i in ngram))
        tmp = word_grams[0]
        n_grams.append(str(tmp))
    articles['temp'] = pd.Series(n_grams) #make new field 'ngrams'. Ngrams are the different sequences of length n in articles.
    n_grams = []
    #return articles with a field ngrams
    for title in articles['Title']:
        n = 2
        title = title.split(' ')
        title = [x.strip().replace('-', '') for x in title]
        title = [x.strip().replace('?', '') for x in title]
        title = [x.strip().replace(':', '') for x in title]
        title = [x.strip().replace('/', '') for x in title]
        title = [x.strip().replace('.', '') for x in title]
        title = [x.strip().replace(',', '') for x in title]
        title = [x.strip().replace('\\', '') for x in title]
        title = [x.strip().replace('(', '') for x in title]
        title = [x.strip().replace(')', '') for x in title]
        title = [x.strip().replace("'", '') for x in title]
        title = [x.strip().replace('"', '') for x in title]
        word_grams = []
        ngram = ngrams(title,n)
        word_grams.append(' '.join(str(i) for i in ngram))
        tmp2 = word_grams[0]
        n_grams.append(str(tmp2))
    articles['temp2'] = pd.Series(n_grams) 
getNgrams(articles)
articles['ngrams'] = articles['temp'] + articles['temp2']
articles = articles.drop(columns=['temp', 'temp2'])
articles.head()

,Content,Title,article_id,ngrams
0,Are you ready to go back to the Moon? NASA to...,LAMP Educational Site,0,"('Are', 'you') ('you', 'ready') ('ready', 'to'..."
1,WASHINGTON - US Secretary of State Hillary Cli...,Clinton urges Egypt Israel to talk on Sinai,1,"('WASHINGTON', '') ('', 'US') ('US', 'Secretar..."
2,"The ""hurricane ride-out team"" at NASA's Kenned...",SPACE.com -- Bracing for Impact,2,"('The', 'hurricane') ('hurricane', 'rideout') ..."
3,Search the history of over 284 billion web pag...,Internet Archive Wayback Machine,3,"('Search', 'the') ('the', 'history') ('history..."
4,The Islamic State (IS) Sunni radical group ove...,IS overruns parts of Unesco-listed Syrian city,4,"('The', 'Islamic') ('Islamic', 'State') ('Stat..."


## Convert n-grams into binary vector representation for each document. You can do some optimzations if the matrix is too big

In [5]:
articles['ngrams'] = articles['ngrams'].str.lower()
articles['ngrams'] = articles['ngrams'].str.replace("', '",' ').str.split("'\) \('")
for i,x in enumerate(articles['ngrams']):
    articles['ngrams'][i][0] = articles['ngrams'][i][0].replace("('",'')
    articles['ngrams'][i][len(articles['ngrams'][i])-1] = articles['ngrams'][i][len(articles['ngrams'][i])-1].replace("')",'')
print(articles.ngrams[0][0])

are you


In [6]:
from  scipy.sparse import csr_matrix

docs = articles.index
indptr = [0] 
indices = [] 
data = [] 
vocabulary = {} 
for d in docs:
    for ngram in articles.ngrams[d]:
        index = vocabulary.setdefault(ngram, len(vocabulary)) 
        indices.append(index) 
        data.append(1) 
    indptr.append(len(indices))
binaryMatrix = csr_matrix((data, indices, indptr), dtype=int)
print(binaryMatrix.shape)


(34565, 3735880)


In [7]:
from __future__ import division

binaryMatrixTransposed = binaryMatrix.transpose()
print(binaryMatrixTransposed.shape)

(3735880, 34565)


## We need hash function that maps integers 0, 1, . . . , k − 1 to bucket numbers 0 through k − 1. It might be impossible to avoid collisions but as long as the collions are too many it won't matter much.

* The simplest would be using the builtin hash() function, it can be for example, hash(rownumber) % Numberofbuckets
* You can generate several of these hash functions by xoring a random integer (hash(rownumber)^randint) % Numberofbuckets
* It can also be a as simple as (rownumber * randint) % Numberofbuckets

In [8]:
import random
import numpy as np

def getHashFunctionValues(numrows, numhashfunctions):
    #return a matrix with hash values 
    numBuckets = numrows 
    hashValMatrix = np.empty((numrows, numhashfunctions)) 
    rand_int = [random.randint(1, numrows) for i in range(0,numhashfunctions)]
    for i in range(0,numrows):
        hashValMatrix[i,:] = [(i^rand_int[j]) % numBuckets for j in range(numhashfunctions)] 
    return hashValMatrix

hashValMatrix=getHashFunctionValues(numrows=binaryMatrixTransposed.shape[0], numhashfunctions=150)

In [9]:
print(hash_val_matrix.shape)

(3735880, 100)


## Compute minhash following the faster algorithm from the lecture 

In [ ]:
def getMinHashSignatureMatrix(binary_matrix, hash_val_matrix)
    #return minhash signature matrix

## Hash signature bands into buckets. Find a way to combine all the signature values in a band and hash them into a number of buckets ususally very high.
* Easiest way is to add all the signature values in the bucket and use a similar hash function like before

In [ ]:
def getLSH(signature_matrix, hashfunctions, num_bands, num_buckets):
   #return lsh buckets or hash table

## Tune parameters to make sure the threshold is appropriate.
## plot the probability of two similar items falling in same bucked for different threshold values


## Choose the best parameters and get nearest neighbors of each articles

## Write the results to submissions.csv file and get the score

In [ ]:
#write article ids, nearest neighbours (documents in same buckets)

#DUE 18.03